In [1]:
# imports
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import os
import sys
import importlib

from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt


sys.path.append("..") # not good
from tools.data import load_data_simc_v1, load_data_radioml_v1

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"


2023-05-13 16:39:03.504923: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-13 16:39:03.531012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 16:39:04.026000: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-13 16:39:05.125997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

## Util functions

In [2]:
def set_seed(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

def reload_module(m: str):
    importlib.reload(sys.modules[m])


## Data loading

In [3]:
# Conatans
RADIO_ML_2016_A_DIR = "/development/data/RML2016.10a/RML2016.10a_dict.pkl" # radioml dataset
RADIO_ML_2016_B_DIR = "/development/data/RML2016.10b/RML2016.10b.dat" # radioml dataset
TRAIN_DATA_DIR = RADIO_ML_2016_B_DIR

MODEL_DATA_TYPE = np.float32

SEED = 123456
set_seed(SEED)

In [4]:
# Loading itself
labels, data, modulations = load_data_radioml_v1(TRAIN_DATA_DIR, to_1024=False)
print(modulations)
print(labels.shape)
print(data.shape)

['PAM4', 'QPSK', '8PSK', 'QAM64', 'BPSK', 'CPFSK', 'WBFM', 'QAM16', 'GFSK', 'AM-DSB']
(1200000,)
(1200000, 1, 128, 2)


In [5]:
# Data Ssplitting
DS_SIZE = len(labels)

n_train = int(0.8 * DS_SIZE)                # 80% - Train
n_validation = int(0.1 * DS_SIZE)           # 10% - Validation
n_test = DS_SIZE - n_train - n_validation   # 10% - Test

random_indecies = np.arange(DS_SIZE)
np.random.shuffle(random_indecies)

train_indecies, validation_indecies, test_indecies, _ = np.split(
    random_indecies, [int(DS_SIZE * 0.8), int(DS_SIZE * 0.9), DS_SIZE]
)

train_data, train_labels = data[train_indecies], labels[train_indecies]
validation_data, validation_labels = data[validation_indecies], labels[validation_indecies]
test_data, test_labels = data[test_indecies], labels[test_indecies]

In [7]:
# Should save some memory
del data
del labels

## Model creation
#### Parts of Encoder

Some utilities for final model


In [8]:
from typing import List
from dataclasses import dataclass

@dataclass
class ConvolutionConfiguration:
    output_channels: List[int]
    kernel_sizes: List[int]
    paddings: List[str]
    max_pool_sizes: List[int]

@dataclass
class DenseConfiguration:
    sizes: List[int]

Model parts

In [9]:
# https://github.com/dksakkos/BatchNorm
class CustomBatchNorm(layers.Layer):
    def __init__(self, *args, **kwargs):
        super(CustomBatchNorm, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.beta = self.add_weight(
            name="custom_batch_beta",
            shape=(input_shape[-1]),
            initializer="zeros",
            trainable=True,
        )

        self.gamma = self.add_weight(
            name="custom_batch_gamma",
            shape=(input_shape[-1]),
            initializer="ones",
            trainable=True,
        )

        self.moving_mean = self.add_weight(
            name="custom_batch_moving_mean",
            shape=(input_shape[-1]),
            initializer=tf.initializers.zeros,
            trainable=False,
        )

        self.moving_variance = self.add_weight(
            name="custom_batch_moving_variance",
            shape=(input_shape[-1]),
            initializer=tf.initializers.ones,
            trainable=False,
        )

### Model
Model constists of a CNNn -> b EncoderLayers -> c Dense layers

In [10]:
def create_model_cnn(
    cnn_conf: ConvolutionConfiguration,
    dense_conf: DenseConfiguration,
    avg_size: int = 32,
    *args, 
    **kwargs
):
    InputLayer = keras.Input(shape=(1, 128, 2))
    assert (
        len(cnn_conf.output_channels)
        == len(cnn_conf.kernel_sizes)
        == len(cnn_conf.paddings)
        == len(cnn_conf.max_pool_sizes)
    )
    N_CNNs = len(cnn_conf.output_channels)
    assert N_CNNs > 0

    CNN1 = layers.Conv2D(
        cnn_conf.output_channels[0],
        (1, cnn_conf.kernel_sizes[0]),
        padding=cnn_conf.paddings[0],
        name="CNN1_",
    )(InputLayer)
    model_layers = [CNN1]
    if cnn_conf.max_pool_sizes[0] != 1:
        model_layers.append(
            layers.MaxPool2D(pool_size=(1, cnn_conf.max_pool_sizes[0]), strides=(1, 2), name=f"MAX_POOL_1_")(
                model_layers[-1]
            )
        )
    model_layers.append(CustomBatchNorm(name="BN1_")(model_layers[-1]))
    model_layers.append(layers.ReLU(name="CNN_RELU1_")(model_layers[-1]))

    for i in range(1, N_CNNs):
        model_layers.append(
            layers.Conv2D(
                cnn_conf.output_channels[i],
                (1, cnn_conf.kernel_sizes[i]),
                padding=cnn_conf.paddings[i],
                name=f"CNN{i+1}_",
            )(model_layers[-1])
        )
        if cnn_conf.max_pool_sizes[i] != 1:
            model_layers.append(
                layers.MaxPool2D(
                    pool_size=(1, cnn_conf.max_pool_sizes[i]),
                    strides=(1, 2),
                    name=f"MAX_POOL_{i+1}_",
                )(model_layers[-1])
            )
        model_layers.append(CustomBatchNorm(name=f"BN{i+1}_")(model_layers[-1]))
        model_layers.append(layers.ReLU(name=f"CNN_RELU{i+1}_")(model_layers[-1]))

    model_layers.append(layers.AveragePooling2D((1, avg_size), name="AVG1_")(model_layers[-1]))

    model_layers.append(layers.Flatten(name="FLT1_")(model_layers[-1]))

    N_Dense = len(dense_conf.sizes)

    if N_Dense > 0:
        model_layers.append(layers.Dense(dense_conf.sizes[0], name="FC1_")(model_layers[-1]))
        model_layers.append(layers.ReLU(name="FC_RELU1_")(model_layers[-1]))

        for i in range(1, N_Dense):
            model_layers.append(
                layers.Dense(dense_conf.sizes[i], name=f"FC{i+1}_")(model_layers[-1])
            )
            model_layers.append(layers.ReLU(name=f"FC_RELU{i+1}_")(model_layers[-1]))

    model_layers.append(layers.Dense(len(modulations), name=f"FC_{N_Dense+1}_")(model_layers[-1]))
    SoftMax = layers.Softmax()(model_layers[-1])
    Output = layers.Flatten()(SoftMax)

    # if args and isinstance(args[0], str):
    #     model = keras.Model(inputs=[InputLayer], outputs=[Output], name=args[0])
    # else:
    model = keras.Model(inputs=[InputLayer], outputs=[Output])
    return model

Test code above

In [11]:
model = create_model_cnn(
    ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192], kernel_sizes=[8, 8, 8, 8, 8, 8], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]),
    DenseConfiguration(sizes=[])
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 128, 2)]       0         
                                                                 
 CNN1_ (Conv2D)              (None, 1, 128, 32)        544       
                                                                 
 BN1_ (CustomBatchNorm)      (None, 1, 128, 32)        128       
                                                                 
 CNN_RELU1_ (ReLU)           (None, 1, 128, 32)        0         
                                                                 
 CNN2_ (Conv2D)              (None, 1, 128, 48)        12336     
                                                                 
 BN2_ (CustomBatchNorm)      (None, 1, 128, 48)        192       
                                                                 
 CNN_RELU2_ (ReLU)           (None, 1, 128, 48)        0     

2023-05-12 02:53:10.332931: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-12 02:53:10.333135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-12 02:53:10.333209: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### List of models we're interested at

In [16]:
default_cnn_conf = ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192], kernel_sizes=[8, 8, 8, 8, 8, 8], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1])
default_dense_conf = DenseConfiguration(sizes=[])
default_avg_size = 32

configs = [
    [default_cnn_conf, default_dense_conf, default_avg_size, "Default_medium"],

    # # Different kernel sizes
    # [ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192], kernel_sizes=[3, 3, 3, 3, 3, 3], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "k=3"],
    # [ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192], kernel_sizes=[17, 17, 17, 17, 17, 17], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "k=17"],
    # [ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192], kernel_sizes=[33, 33, 33, 33, 33, 33], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "k=33"],
    # [ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192], kernel_sizes=[65, 65, 65, 65, 65, 65], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "k=65"],

    # # Different Width of encoder layers
    # [ConvolutionConfiguration(output_channels=[16, 16, 32, 32, 64, 64], kernel_sizes=[8, 8, 8, 8, 8, 8], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "output_channels=[16, 16, 32, 32, 64, 64]"],
    # [ConvolutionConfiguration(output_channels=[32, 32, 48, 64, 64, 96], kernel_sizes=[8, 8, 8, 8, 8, 8], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "output_channels=[32, 32, 48, 64, 64, 96]"],
    # [ConvolutionConfiguration(output_channels=[64, 64, 128, 192, 192, 256], kernel_sizes=[8, 8, 8, 8, 8, 8], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "output_channels=[64, 64, 128, 192, 192, 256]"],


    # # Different depth of model
    # [ConvolutionConfiguration(output_channels=[32, 48, 64], kernel_sizes=[8, 8, 8], paddings=["same", "same", "same"], max_pool_sizes=[1, 2, 2]), default_dense_conf, default_avg_size, "output_channels=[32, 48, 64]__max_pool_sizes=[1, 2, 2]"],
    [ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192, 256, 512], kernel_sizes=[8, 8, 8, 8, 8, 8, 8, 8], paddings=["same", "same", "same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1, 2, 1]), default_dense_conf, 16, "output_channels=[32, 48, 64, 96, 128, 192, 256, 512]__max_pool_sizes=[1, 1, 2, 1, 2, 1, 2, 1]__avg_size=16"],
]

# for config in configs:
#     print(config)
#     model = create_model_cnn(*config)
#     model.summary()

### Model evaluation functions

In [17]:
import pickle

def evaluate_model(model, data, labels, verbose=True):
    preds = model.predict(data)
    pred_labels = np.argmax(preds, axis=1)

    cls_to_acc = {"Overall": accuracy_score(labels, pred_labels)}
    print(f"Overall test accuracy: {cls_to_acc}")
    for ci, cl in enumerate(modulations):
        class_indecies = np.where(labels == ci)[0]
        cur_true_labels = labels[class_indecies]
        cur_pred_labels = pred_labels[class_indecies]
        cls_to_acc[cl] = accuracy_score(cur_true_labels, cur_pred_labels)
        verbose and print(f"{cl} test accuracy: {cls_to_acc[cl]}")
    cm = confusion_matrix(y_true=labels, y_pred=pred_labels)
    verbose and print(cm)
    df_cm = pd.DataFrame(cm, index = modulations,
                    columns = modulations)
    if verbose:
        plt.figure(figsize = (10,7))
        sn.heatmap(df_cm, annot=True)
    return cm, cls_to_acc


def get_snrs(indecies, to_1024=False, minimum_snr=-100):
    with open(TRAIN_DATA_DIR, 'rb') as crmrn_file:
        raw_ds = pickle.load(crmrn_file, encoding="bytes")
    decoded_raw_ds = {}
    for (class_name_bytes, snr), raw_data in raw_ds.items():
        if snr < minimum_snr:
            continue
        decoded_raw_ds[(class_name_bytes.decode("utf-8"), snr)] = raw_data
    raw_ds = decoded_raw_ds

    samples_per_snr_per_modulation = raw_ds[list(raw_ds.keys())[0]].shape[0] # 1000
    snrs_sequence = [k[1] for k in raw_ds.keys()]
    result = []
    snrs = [snrs_sequence[idx // samples_per_snr_per_modulation] for idx in indecies]
    return np.array(snrs)


def get_snr_to_acc(model, data, labels, snrs, verbose=True):
    snr_to_acc = {}
    for snr in range(min(snrs), max(snrs)+2, 2):
        cur_indecies = np.where(snrs == snr)[0]
        cur_data = data[cur_indecies]
        cur_labels = labels[cur_indecies]
        cur_pred = model.predict(cur_data, verbose=0)

        cur_pred_labels = np.argmax(cur_pred, axis=1)
        verbose and print(f"snr={snr}")
        acc = accuracy_score(cur_labels, cur_pred_labels)
        verbose and print(f"Overall test accuracy: {acc}")
        snr_to_acc[snr] = acc
    return snr_to_acc

### Train, evaluate, save model and evaluation results

In [18]:
import json
from copy import deepcopy

def step_decay(epoch):
    lrate = 0.001
    factor = epoch // 8
    lrate /= (10**factor)
    return lrate

lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

train_data = train_data.squeeze()
validation_data = validation_data.squeeze()
test_data = test_data.squeeze()

train_data = np.expand_dims(train_data, 1)
validation_data = np.expand_dims(validation_data, 1)
test_data = np.expand_dims(test_data, 1)

# configs = [[default_cnn_conf, default_encoder_conf, default_dense_conf, default_avg_size],
#    [default_cnn_conf, default_encoder_conf, default_dense_conf, 64]]
# configs = [
#     [default_cnn_conf, default_dense_conf, default_avg_size, "Default_medium"],
#     [ConvolutionConfiguration(output_channels=[32, 48, 64, 96, 128, 192], kernel_sizes=[17, 17, 17, 17, 17, 17], paddings=["same", "same", "same", "same", "same", "same"], max_pool_sizes=[1, 1, 2, 1, 2, 1]), default_dense_conf, default_avg_size, "k=17"],
# ]

for i, config in enumerate(configs):
    model = create_model_cnn(*config)
    model.summary()

    model.compile(
        # optimizer=tf.keras.optimizers.Adam(learning_rate=0.0),
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    N_EPOCHS = 16
    BATCH_SIZE = 256

    h = model.fit(
        train_data,
        train_labels,
        epochs=N_EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(validation_data, validation_labels),
        callbacks=[lrate]
    )

    cm_val, cls_to_acc_val = evaluate_model(
        model, validation_data, validation_labels, verbose=False
    )
    snrs_val = get_snrs(validation_indecies, False)
    snr_to_acc_val = get_snr_to_acc(
        model, validation_data, validation_labels, snrs_val, verbose=False
    )

    cm_test, cls_to_acc_test = evaluate_model(model, test_data, test_labels, verbose=False)
    snr_tests = get_snrs(test_indecies, False)
    snr_to_acc_test = get_snr_to_acc(model, test_data, test_labels, snr_tests, verbose=False)

    results_dir = f"CNN_radio_ML/experiment_{i}/"
    model_configuration = {
        "cnn": config[0].__dict__,
        "dense": config[1].__dict__,
        "avg_pool": config[2],
    }
    train_history = deepcopy(h.history)
    train_history["lr"] = [float(lr) for lr in train_history["lr"]]

    
    results = {
        "model_configuration": model_configuration,
        "train_history": train_history,
        "cm_val": cm_val.tolist(),
        "cls_to_acc_val": cls_to_acc_val,
        "snr_to_acc_val": snr_to_acc_val,
        "cm_test": cm_test.tolist(),
        "cls_to_acc_test": cls_to_acc_test,
        "snr_to_acc_test": snr_to_acc_test,
    }

    os.makedirs(results_dir, exist_ok=True)
    with open(results_dir + "results.json", "w") as res_file:
        json.dump(results, res_file, indent=4)
    model.save(results_dir + "model")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 128, 2)]       0         
                                                                 
 CNN1_ (Conv2D)              (None, 1, 128, 32)        544       
                                                                 
 BN1_ (CustomBatchNorm)      (None, 1, 128, 32)        128       
                                                                 
 CNN_RELU1_ (ReLU)           (None, 1, 128, 32)        0         
                                                                 
 CNN2_ (Conv2D)              (None, 1, 128, 48)        12336     
                                                                 
 BN2_ (CustomBatchNorm)      (None, 1, 128, 48)        192       
                                                                 
 CNN_RELU2_ (ReLU)           (None, 1, 128, 48)        0   

2023-05-12 02:55:32.474977: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.59GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-12 02:55:32.969423: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.60GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-12 02:55:33.016746: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.13GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-12 02:55:33.306421: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU

   8/3750 [..............................] - ETA: 27s - loss: 2.3019 - accuracy: 0.1089    

2023-05-12 02:55:33.970460: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


3750/3750 [==============================] - 34s 8ms/step - loss: 1.2385 - accuracy: 0.4886 - val_loss: 1.0558 - val_accuracy: 0.5626 - lr: 0.0010
Epoch 2/16
3750/3750 [==============================] - 31s 8ms/step - loss: 1.0245 - accuracy: 0.5729 - val_loss: 1.0051 - val_accuracy: 0.5854 - lr: 0.0010
Epoch 3/16
3750/3750 [==============================] - 31s 8ms/step - loss: 0.9881 - accuracy: 0.5966 - val_loss: 0.9747 - val_accuracy: 0.6064 - lr: 0.0010
Epoch 4/16
3750/3750 [==============================] - 32s 8ms/step - loss: 0.9611 - accuracy: 0.6122 - val_loss: 0.9538 - val_accuracy: 0.6161 - lr: 0.0010
Epoch 5/16
3750/3750 [==============================] - 32s 8ms/step - loss: 0.9407 - accuracy: 0.6215 - val_loss: 0.9364 - val_accuracy: 0.6234 - lr: 0.0010
Epoch 6/16
3750/3750 [==============================] - 32s 9ms/step - loss: 0.9265 - accuracy: 0.6279 - val_loss: 0.9331 - val_accuracy: 0.6266 - lr: 0.0010
Epoch 7/16
3750/3750 [==============================] - 32s 9ms

...

INFO:tensorflow:Assets written to: CNN_radio_ML/experiment_0/model/assets


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1, 128, 2)]       0         
                                                                 
 CNN1_ (Conv2D)              (None, 1, 128, 32)        544       
                                                                 
 BN1_ (CustomBatchNorm)      (None, 1, 128, 32)        128       
                                                                 
 CNN_RELU1_ (ReLU)           (None, 1, 128, 32)        0         
                                                                 
 CNN2_ (Conv2D)              (None, 1, 128, 48)        12336     
                                                                 
 BN2_ (CustomBatchNorm)      (None, 1, 128, 48)        192       
                                                                 
 CNN_RELU2_ (ReLU)           (None, 1, 128, 48)        0   

3750/3750 [==============================] - 57s 15ms/step - loss: 1.2853 - accuracy: 0.4726 - val_loss: 1.0476 - val_accuracy: 0.5630 - lr: 0.0010
Epoch 2/16
3750/3750 [==============================] - 55s 15ms/step - loss: 1.0177 - accuracy: 0.5728 - val_loss: 1.0001 - val_accuracy: 0.5825 - lr: 0.0010
Epoch 3/16
3750/3750 [==============================] - 55s 15ms/step - loss: 0.9883 - accuracy: 0.5909 - val_loss: 0.9844 - val_accuracy: 0.5971 - lr: 0.0010
Epoch 4/16
3750/3750 [==============================] - 55s 15ms/step - loss: 0.9628 - accuracy: 0.6086 - val_loss: 0.9551 - val_accuracy: 0.6136 - lr: 0.0010
Epoch 5/16
3750/3750 [==============================] - 55s 15ms/step - loss: 0.9392 - accuracy: 0.6217 - val_loss: 0.9429 - val_accuracy: 0.6206 - lr: 0.0010
Epoch 6/16
3750/3750 [==============================] - 55s 15ms/step - loss: 0.9224 - accuracy: 0.6291 - val_loss: 0.9372 - val_accuracy: 0.6258 - lr: 0.0010
Epoch 7/16
3750/3750 [==============================] - 5

INFO:tensorflow:Assets written to: CNN_radio_ML/experiment_1/model/assets


INFO:tensorflow:Assets written to: CNN_radio_ML/experiment_1/model/assets


In [29]:
print(h.history)
print(type(h.history["loss"][0]))
print(type(h.history["accuracy"][0]))
print(type(h.history["val_loss"][0]))
print(type(h.history["val_accuracy"][0]))
print(type(h.history["lr"][0]))

{'loss': [1.6604399681091309], 'accuracy': [0.3758068084716797], 'val_loss': [1.3640673160552979], 'val_accuracy': [0.4716818034648895], 'lr': [0.001]}
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'numpy.float32'>


In [20]:
with open(results_dir + "results.json", "w") as res_file:
    json.dump(results, res_file, indent=4)